In [502]:
import json
import requests
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup

In [503]:
#https://www.kaggle.com/datasets/max-mind/world-cities-database?resource=download&select=worldcitiespop.csv
dfCiudades = pd.read_csv('worldcities.csv')

In [504]:
#Imprimir las 5 primeras instacias
dfCiudades.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


In [505]:
#Esquina superior derecha 38.75, 90.92
#Esquina inferior derecha -52.31 90.92
#Esquina superior izquierda 38.75, 179.43
#Esquina inferior izquierda -52.31 179.

dfCiudadesFiltered = dfCiudades[(dfCiudades['lat']< 50) & (dfCiudades['lat']>-60.31) & (dfCiudades['lng'] >60) & (dfCiudades['lng'] < 180) ]
dfCiudadesFiltered.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


In [506]:
dfCiudadesFiltered.shape

(14651, 11)

In [507]:
dfCiudadesFiltered = dfCiudadesFiltered.drop(columns = ['city_ascii','iso2','iso3','admin_name','id', 'capital'])
dfCiudadesFiltered.head()

,city,lat,lng,country,population
0,Tokyo,35.6897,139.6922,Japan,37732000.0
1,Jakarta,-6.1750,106.8275,Indonesia,33756000.0
2,Delhi,28.6100,77.2300,India,32226000.0
3,Guangzhou,23.1300,113.2600,China,26940000.0
4,Mumbai,19.0761,72.8775,India,24973000.0


In [508]:
# Inicializar el WebDriver
driver = webdriver.Chrome()

# Navegar a la página web que contiene la 
driver.get('https://www.numbeo.com/crime/region_rankings.jsp?title=2024&region=142')

# Obtener el contenido HTML de la página
html = driver.page_source

# Cerrar el navegador
driver.quit()

# Usar BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Encontrar la tabla que queremos extraer
tabla = soup.find('table', {'id': 't2'})

# Extraer los encabezados de la tabla
headers = []
for th in tabla.find_all('th'):
    headers.append(th.text.strip())

# Extraer los datos de las filas de la tabla
rows = []
for tr in tabla.find_all('tr')[1:]:  # Omitimos la primera fila de encabezados
    cells = []
    for td in tr.find_all('td'):
        cells.append(td.text.strip())
    rows.append(cells)

# Convertir los datos en un DataFrame de pandas
dfCriminalidad = pd.DataFrame(rows, columns=headers)

In [509]:
dfCriminalidad = dfCriminalidad.drop(columns='Rank')
dfCriminalidad.head()

,City,Crime Index,Safety Index
0,"Damascus, Syria",70.6,29.4
1,"Manila, Philippines",64.7,35.3
2,"Dhaka, Bangladesh",62.7,37.3
3,"Kuala Lumpur, Malaysia",61.8,38.2
4,"Delhi, India",59.4,40.6


In [510]:
ciudad = dfCriminalidad['City'].str.split(', ',expand = True)
ciudad.columns = ['ciudad', 'pais']

dfCriminalidad = pd.concat([dfCriminalidad,ciudad], axis = 1)
dfCriminalidad = dfCriminalidad.drop(columns=['City'])
dfCriminalidad.head()

,Crime Index,Safety Index,ciudad,pais
0,70.6,29.4,Damascus,Syria
1,64.7,35.3,Manila,Philippines
2,62.7,37.3,Dhaka,Bangladesh
3,61.8,38.2,Kuala Lumpur,Malaysia
4,59.4,40.6,Delhi,India


In [511]:
# Inicializar el WebDriver
driver = webdriver.Chrome()

# Navegar a la página web que contiene la tabla
driver.get('https://www.numbeo.com/crime/region_rankings.jsp?title=2024&region=009')

# Obtener el contenido HTML de la página
html = driver.page_source

# Cerrar el navegador
driver.quit()

# Usar BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Encontrar la tabla que queremos extraer
tabla = soup.find('table', {'id': 't2'})

# Extraer los encabezados de la tabla
headers = []
for th in tabla.find_all('th'):
    headers.append(th.text.strip())

# Extraer los datos de las filas de la tabla
rows = []
for tr in tabla.find_all('tr')[1:]:  # Omitimos la primera fila de encabezados
    cells = []
    for td in tr.find_all('td'):
        cells.append(td.text.strip())
    rows.append(cells)

# Convertir los datos en un DataFrame de pandas
dfCriminalidad2 = pd.DataFrame(rows, columns=headers)

In [512]:
dfCriminalidad2 = dfCriminalidad2.drop(columns='Rank')
dfCriminalidad2.head()

,City,Crime Index,Safety Index
0,"Port Moresby, Papua New Guinea",80.6,19.4
1,"Alice Springs, Australia",70.2,29.8
2,"Auckland, New Zealand",49.6,50.4
3,"Gold Coast, Australia",46.9,53.1
4,"Christchurch, New Zealand",44.8,55.2


In [513]:
ciudad = dfCriminalidad2['City'].str.split(',',expand = True)
ciudad.columns = ['ciudad', 'pais']

dfCriminalidad2 = pd.concat([dfCriminalidad2,ciudad], axis = 1)
dfCriminalidad2 = dfCriminalidad2.drop(columns=['City'])
dfCriminalidad2.head()

,Crime Index,Safety Index,ciudad,pais
0,80.6,19.4,Port Moresby,Papua New Guinea
1,70.2,29.8,Alice Springs,Australia
2,49.6,50.4,Auckland,New Zealand
3,46.9,53.1,Gold Coast,Australia
4,44.8,55.2,Christchurch,New Zealand


In [514]:
cont = 0
for index, fila in dfCiudadesFiltered.iterrows():
    for index2, fila2 in dfCriminalidad.iterrows():
        if(fila['city']==fila2['ciudad']):
            cont+=1

print(cont)

38


In [515]:
cont = 0
for index, fila in dfCiudadesFiltered.iterrows():
    for index2, fila2 in dfCriminalidad2.iterrows():
        if(fila['city']==fila2['ciudad']):
            cont+=1

print(cont)

14


In [516]:
dfCriminalidad = pd.concat([dfCriminalidad,dfCriminalidad2])
dfCriminalidad.shape

(89, 4)

In [517]:
for index, fila in dfCiudadesFiltered.iterrows():
    for index2, fila2 in dfCriminalidad.iterrows():
        if(fila['city']==fila2['ciudad']):
            dfCiudadesFiltered.at[index, 'Crime Index'] = fila2['Crime Index']
            dfCiudadesFiltered.at[index, 'Safety Index'] = fila2['Safety Index']

In [518]:
dfCiudadesFiltCrim = dfCiudadesFiltered.dropna()
dfCiudadesFiltCrim.shape

(52, 7)

In [519]:
#Api Key OpenWeather
openWeatherAPIKey = 'your_api_key'

def openWeatherAPICall(lat,lng):
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lng}&appid={openWeatherAPIKey}'
    response = requests.get(url)
    if response.status_code ==200:
        data = response.json()
        return data
    else:
        print('Error')
        return None

In [520]:
with open('ejemploAPIWeather.json', 'r') as archivo:
    datos = json.load(archivo)

print(datos['list'][0]['main'])

{'temp': 281.88, 'feels_like': 281.13, 'temp_min': 281.88, 'temp_max': 281.96, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 939, 'humidity': 97, 'temp_kf': -0.08}


In [521]:
def datosClimaJson(data):
    pressure = 0
    temp = 0
    sea_level = 0
    humidity = 0

    for elemento in data['list']:
        pressure += elemento['main']['pressure']
        temp += elemento['main']['temp']
        sea_level += elemento['main']['sea_level']
        humidity += elemento['main']['humidity']
    
    tam = len(data['list'])
    pressure  = pressure/tam
    temp = temp/tam
    sea_level = sea_level/tam
    humidity = humidity/tam
    return {'pressure':pressure,'temp':temp,'sea_level':sea_level,'humidity':humidity}


In [523]:
import time
# Contador de llamadas
call_count = 0

for index, fila in dfCiudadesFiltCrim.iterrows():
    data = openWeatherAPICall(fila['lat'], fila['lng'])  # Acceder directamente a los valores de la fila
    data = datosClimaJson(data)
    
    # Actualizar el DataFrame original utilizando el índice
    dfCiudadesFiltCrim.at[index, 'pressure'] = data['pressure']
    dfCiudadesFiltCrim.at[index, 'temp'] = data['temp']
    dfCiudadesFiltCrim.at[index, 'sea_level'] = data['sea_level']
    dfCiudadesFiltCrim.at[index, 'humidity'] = data['humidity']
    
    # Incrementar el contador de llamadas
    call_count += 1
    
    # Verificar si hemos alcanzado el límite de 60 llamadas
    if call_count >= 60:
        print("Esperando 60 segundos para continuar...")
        time.sleep(60)  # Pausar durante 60 segundos
        call_count = 0  # Reiniciar el contador

# Ver las primeras filas del DataFrame
dfCiudadesFiltCrimWeath = dfCiudadesFiltCrim
dfCiudadesFiltCrimWeath.tail()

,city,lat,lng,country,population,Crime Index,Safety Index,pressure,temp,sea_level,humidity
1869,Canberra,-35.2931,149.1269,Australia,381488.0,28.4,71.6,1022.800,280.37025,1022.800,56.900
3078,Wellington,-41.2889,174.7772,New Zealand,216200.0,33.8,66.2,999.150,282.76800,999.150,78.325
3345,Hobart,-42.8806,147.3250,Australia,197451.0,33.6,66.4,1015.150,280.75350,1015.150,75.125
19417,Wellington,11.3655,76.7844,India,26686.0,33.8,66.2,1012.225,289.96700,1012.225,64.825
21526,Alice Springs,-23.7000,133.8700,Australia,23726.0,70.2,29.8,1020.775,291.01725,1020.775,25.200


In [524]:
dfCiudadesFiltCrimWeath.shape

(52, 11)

In [525]:
# Inicializar el WebDriver
driver = webdriver.Chrome()

# Navegar a la página web que contiene la tabla
driver.get('https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2024&region=142')

# Obtener el contenido HTML de la página
html = driver.page_source

# Cerrar el navegador
driver.quit()

# Usar BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Encontrar la tabla que queremos extraer
tabla = soup.find('table', {'id': 't2'})

# Extraer los encabezados de la tabla
headers = []
for th in tabla.find_all('th'):
    headers.append(th.text.strip())

# Extraer los datos de las filas de la tabla
rows = []
for tr in tabla.find_all('tr')[1:]:  # Omitimos la primera fila de encabezados
    cells = []
    for td in tr.find_all('td'):
        cells.append(td.text.strip())
    rows.append(cells)

# Convertir los datos en un DataFrame de pandas
dfCostLiving = pd.DataFrame(rows, columns=headers)
dfCostLiving.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,"Singapore, Singapore",81.9,71.9,77.1,75.0,55.3,103.0
1,2,"Tel Aviv-Yafo, Israel",72.5,42.3,58.2,66.1,79.3,81.6
2,3,"Hong Kong, Hong Kong (China)",71.5,61.4,66.7,79.6,47.0,84.1
3,4,"Seoul, South Korea",70.3,29.1,50.8,92.1,37.4,91.3
4,5,"Jerusalem, Israel",66.1,38.5,53.0,59.3,70.2,72.3


In [526]:
dfCostLiving = dfCostLiving.drop(columns = 'Rank')

In [527]:
ciudad = dfCostLiving['City'].str.split(',',expand = True)
ciudad.columns = ['ciudad', 'pais','a']

dfCostLiving = pd.concat([dfCostLiving,ciudad], axis = 1)
dfCostLiving = dfCostLiving.drop(columns=['City', 'a'])
dfCostLiving.head()

,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,ciudad,pais
0,81.9,71.9,77.1,75.0,55.3,103.0,Singapore,Singapore
1,72.5,42.3,58.2,66.1,79.3,81.6,Tel Aviv-Yafo,Israel
2,71.5,61.4,66.7,79.6,47.0,84.1,Hong Kong,Hong Kong (China)
3,70.3,29.1,50.8,92.1,37.4,91.3,Seoul,South Korea
4,66.1,38.5,53.0,59.3,70.2,72.3,Jerusalem,Israel


In [528]:
cont = 0
for index, fila in dfCiudadesFiltCrimWeath.iterrows():
    for index2, fila2 in dfCostLiving.iterrows():
        if(fila['city']==fila2['ciudad']):
            cont+=1

print(cont)

33


In [529]:
# Inicializar el WebDriver
driver = webdriver.Chrome()

# Navegar a la página web que contiene la tabla
driver.get('https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2024&region=009')

# Obtener el contenido HTML de la página
html = driver.page_source

# Cerrar el navegador
driver.quit()

# Usar BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Encontrar la tabla que queremos extraer
tabla = soup.find('table', {'id': 't2'})

# Extraer los encabezados de la tabla
headers = []
for th in tabla.find_all('th'):
    headers.append(th.text.strip())

# Extraer los datos de las filas de la tabla
rows = []
for tr in tabla.find_all('tr')[1:]:  # Omitimos la primera fila de encabezados
    cells = []
    for td in tr.find_all('td'):
        cells.append(td.text.strip())
    rows.append(cells)

# Convertir los datos en un DataFrame de pandas
dfCostLiving2 = pd.DataFrame(rows, columns=headers)
dfCostLiving2.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,"Canberra, Australia",87.5,42.4,66.1,80.8,105.7,66.2
1,2,"Adelaide, Australia",81.7,35.0,59.6,80.5,87.8,72.2
2,3,"Sydney, Australia",75.9,55.4,66.2,76.7,63.3,86.2
3,4,"Perth, Australia",75.6,38.8,58.2,76.9,69.9,97.1
4,5,"Wellington, New Zealand",73.3,40.1,57.5,76.5,63.1,68.2


In [530]:
dfCostLiving2 = dfCostLiving2.drop(columns = 'Rank')

In [531]:
ciudad = dfCostLiving2['City'].str.split(',',expand = True)
ciudad.columns = ['ciudad', 'pais']

dfCostLiving2 = pd.concat([dfCostLiving2,ciudad], axis = 1)
dfCostLiving2 = dfCostLiving2.drop(columns=['City'])
dfCostLiving2.head()

,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,ciudad,pais
0,87.5,42.4,66.1,80.8,105.7,66.2,Canberra,Australia
1,81.7,35.0,59.6,80.5,87.8,72.2,Adelaide,Australia
2,75.9,55.4,66.2,76.7,63.3,86.2,Sydney,Australia
3,75.6,38.8,58.2,76.9,69.9,97.1,Perth,Australia
4,73.3,40.1,57.5,76.5,63.1,68.2,Wellington,New Zealand


In [532]:
cont = 0
for index, fila in dfCiudadesFiltCrimWeath.iterrows():
    for index2, fila2 in dfCostLiving2.iterrows():
        if(fila['city'] == fila2['ciudad']):
            cont+=1

print(cont)

10


In [533]:
dfCostLiving = pd.concat([dfCostLiving,dfCostLiving2])
dfCostLiving.head()

,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,ciudad,pais
0,81.9,71.9,77.1,75.0,55.3,103.0,Singapore,Singapore
1,72.5,42.3,58.2,66.1,79.3,81.6,Tel Aviv-Yafo,Israel
2,71.5,61.4,66.7,79.6,47.0,84.1,Hong Kong,Hong Kong (China)
3,70.3,29.1,50.8,92.1,37.4,91.3,Seoul,South Korea
4,66.1,38.5,53.0,59.3,70.2,72.3,Jerusalem,Israel


In [534]:
for index, fila in dfCiudadesFiltCrimWeath.iterrows():
    for index2, fila2 in dfCostLiving.iterrows():
        if(fila['city']==fila2['ciudad']):
            dfCiudadesFiltCrimWeath.at[index, 'CCost of Living Index'] = fila2['Cost of Living Index']
            dfCiudadesFiltCrimWeath.at[index, 'Rent Index'] = fila2['Rent Index']
            dfCiudadesFiltCrimWeath.at[index, 'Cost of Living Plus Rent Index'] = fila2['Cost of Living Plus Rent Index']
            dfCiudadesFiltCrimWeath.at[index, 'Groceries Index'] = fila2['Groceries Index']
            dfCiudadesFiltCrimWeath.at[index, 'Restaurant Price Index'] = fila2['Restaurant Price Index']
            dfCiudadesFiltCrimWeath.at[index, 'Local Purchasing Power Index	'] = fila2['Local Purchasing Power Index']

dfCiudadesFiltCrimWeathCostlive = dfCiudadesFiltCrimWeath
dfCiudadesFiltCrimWeathCostlive.shape

C:\Users\jusec\AppData\Local\Temp\ipykernel_24228\799178665.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCiudadesFiltCrimWeath.at[index, 'CCost of Living Index'] = fila2['Cost of Living Index']
C:\Users\jusec\AppData\Local\Temp\ipykernel_24228\799178665.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCiudadesFiltCrimWeath.at[index, 'Rent Index'] = fila2['Rent Index']
C:\Users\jusec\AppData\Local\Temp\ipykernel_24228\799178665.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

(52, 17)

In [535]:
dfCiudadesFiltCrimWeathCostlive = dfCiudadesFiltCrimWeathCostlive.dropna()
dfCiudadesFiltCrimWeathCostlive.head()

,city,lat,lng,country,population,Crime Index,Safety Index,pressure,temp,sea_level,humidity,CCost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index\t
0,Tokyo,35.6897,139.6922,Japan,37732000.0,24.1,75.9,1012.450,302.06875,1012.450,66.575,57.4,29.3,44.1,67.0,35.4,98.6
1,Jakarta,-6.1750,106.8275,Indonesia,33756000.0,52.7,47.3,1008.950,302.25975,1008.950,54.550,32.0,12.8,22.9,36.4,16.9,35.5
2,Delhi,28.6100,77.2300,India,32226000.0,59.4,40.6,1003.250,305.07975,1003.250,47.250,24.4,7.3,16.3,23.8,19.9,93.9
3,Guangzhou,23.1300,113.2600,China,26940000.0,30.4,69.6,1003.175,302.91975,1003.175,69.650,35.1,13.3,24.8,40.2,22.4,70.0
4,Mumbai,19.0761,72.8775,India,24973000.0,44.8,55.2,1008.225,300.96550,1008.225,69.675,25.2,19.0,22.2,25.6,21.0,54.2


In [640]:
def googlePlacesParksApiCall(lat,lng):
    googlePlacesAPIkEY = 'your_api_key'
    location = f"{lat},{lng}"
    radius = 380
    type = "park"

    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={type}&key={googlePlacesAPIkEY}"

    response = requests.get(url)
    data = response.json()
    return data

In [641]:
def googlePlacesRestaurantApiCall(lat,lng):
    googlePlacesAPIkEY = 'your_api_key'
    location = f"{lat},{lng}"
    radius = 135
    type = "restaurant"

    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={type}&key={googlePlacesAPIkEY}"

    response = requests.get(url)
    data = response.json()
    return data

In [642]:
def googlePlacesHospitalsApiCall(lat,lng):
    googlePlacesAPIkEY = 'your_api_key'
    location = f"{lat},{lng}"
    radius = 375
    type = "hospital"

    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={type}&key={googlePlacesAPIkEY}"

    response = requests.get(url)
    data = response.json()
    return data

In [643]:
def googleAirQualityApiCall(lat,lng):
    googlePlacesAPIKey = 'your_api_key'
    url = f'https://airquality.googleapis.com/v1/currentConditions:lookup?key={googlePlacesAPIKey}'
    data = {
        "location": {
            "latitude": lat,
            "longitude": lng
        }
    }

    headers = {
        'Content-Type': 'application/json'
    }

    # Realizar la solicitud POST
    response = requests.post(url, headers=headers, data = json.dumps(data))

    return response.json()

In [644]:
dfCiudadesFiltCrimWeathCostliveAPI = dfCiudadesFiltCrimWeathCostlive
dfCiudadesFiltCrimWeathCostliveAPI.shape

(43, 22)

In [645]:
ciudades = list(dfCiudadesFiltCrimWeathCostliveAPI['city'])
print(ciudades)

['Tokyo', 'Jakarta', 'Delhi', 'Guangzhou', 'Mumbai', 'Manila', 'Shanghai', 'Seoul', 'Dhaka', 'Beijing', 'Bangkok', 'Shenzhen', 'Karachi', 'Bangalore', 'Ho Chi Minh City', 'Chennai', 'Lahore', 'Kuala Lumpur', 'Hanoi', 'Pune', 'Ahmedabad', 'Hong Kong', 'Singapore', 'Melbourne', 'Sydney', 'Taipei', 'Brisbane', 'Colombo', 'Perth', 'Phnom Penh', 'Vadodara', 'Almaty', 'Auckland', 'Adelaide', 'Chiang Mai', 'Islamabad', 'Gurgaon', 'Kathmandu', 'Noida', 'Christchurch', 'Canberra', 'Wellington', 'Wellington']


In [646]:
dfCiudadesFiltCrimWeathCostliveAPI.head()

,city,lat,lng,country,population,Crime Index,Safety Index,pressure,temp,sea_level,...,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index\t,parks,restaurants,hospitals,aqi,dominant Pollutant
0,Tokyo,35.6897,139.6922,Japan,37732000.0,24.1,75.9,1012.450,302.06875,1012.450,...,29.3,44.1,67.0,35.4,98.6,0.0,0.0,1.0,80.0,pm25
1,Jakarta,-6.1750,106.8275,Indonesia,33756000.0,52.7,47.3,1008.950,302.25975,1008.950,...,12.8,22.9,36.4,16.9,35.5,1.0,0.0,0.0,55.0,pm25
2,Delhi,28.6100,77.2300,India,32226000.0,59.4,40.6,1003.250,305.07975,1003.250,...,7.3,16.3,23.8,19.9,93.9,0.0,0.0,0.0,65.0,pm25
3,Guangzhou,23.1300,113.2600,China,26940000.0,30.4,69.6,1003.175,302.91975,1003.175,...,13.3,24.8,40.2,22.4,70.0,0.0,0.0,0.0,61.0,pm25
4,Mumbai,19.0761,72.8775,India,24973000.0,44.8,55.2,1008.225,300.96550,1008.225,...,19.0,22.2,25.6,21.0,54.2,20.0,20.0,20.0,55.0,pm10


In [647]:
aqi = googleAirQualityApiCall(35.6897,139.6922)
print(aqi)

{'dateTime': '2024-09-14T04:00:00Z', 'regionCode': 'jp', 'indexes': [{'code': 'uaqi', 'displayName': 'Universal AQI', 'aqi': 80, 'aqiDisplay': '80', 'color': {'red': 0.34509805, 'green': 0.74509805, 'blue': 0.20784314}, 'category': 'Excellent air quality', 'dominantPollutant': 'pm25'}]}


In [648]:
print(aqi['indexes'][0]['dominantPollutant'])

pm25


In [653]:
dfCiudadesFiltCrimWeathCostliveAPI = dfCiudadesFiltCrimWeathCostliveAPI.drop(columns=['parks','restaurants','hospitals','aqi','dominant Pollutant'])

In [654]:
# Iterar sobre los DataFrames y realizar las llamadas a las APIs
for index, elemento in dfCiudadesFiltCrimWeathCostliveAPI.iterrows():
    parks = googlePlacesParksApiCall(elemento['lat'], elemento['lng'])
    restaurants = googlePlacesRestaurantApiCall(elemento['lat'], elemento['lng'])
    hospitals = googlePlacesHospitalsApiCall(elemento['lat'], elemento['lng'])
    aqi = googleAirQualityApiCall(elemento['lat'], elemento['lng'])

    print(f"Index: {index} | Parks: {len(parks['results']) if 'results' in parks else 'No data'} | Restaurants: {len(restaurants['results']) if 'results' in restaurants else 'No data'} | Hospitals: {len(hospitals['results']) if 'results' in hospitals else 'No data'}")


    # Actualizar los campos en el DataFrame
    dfCiudadesFiltCrimWeathCostliveAPI.at[index, 'parks'] = len(parks['results'])
    dfCiudadesFiltCrimWeathCostliveAPI.at[index, 'restaurants'] = len(restaurants['results'])
    dfCiudadesFiltCrimWeathCostliveAPI.at[index, 'hospitals'] = len(hospitals['results'])

    # Verificar si el objeto 'aqi' tiene la información correcta
    if 'indexes' in aqi and aqi['indexes']:
        dfCiudadesFiltCrimWeathCostliveAPI.at[index, 'aqi'] = aqi['indexes'][0]['aqi']
        dfCiudadesFiltCrimWeathCostliveAPI.at[index, 'dominant Pollutant'] = aqi['indexes'][0]['dominantPollutant']
    else:
        print(f"Información de AQI no disponible para index: {index}")
        dfCiudadesFiltCrimWeathCostliveAPI.at[index, 'aqi'] = None
        dfCiudadesFiltCrimWeathCostliveAPI.at[index, 'dominant Pollutant'] = None

# Mostrar el tamaño del DataFrame actualizado
print(dfCiudadesFiltCrimWeathCostliveAPI.shape)


Index: 0 | Parks: 18 | Restaurants: 6 | Hospitals: 5
Index: 1 | Parks: 4 | Restaurants: 0 | Hospitals: 0
Index: 2 | Parks: 12 | Restaurants: 2 | Hospitals: 1
Index: 3 | Parks: 0 | Restaurants: 20 | Hospitals: 20
Index: 4 | Parks: 20 | Restaurants: 20 | Hospitals: 20
Index: 5 | Parks: 16 | Restaurants: 4 | Hospitals: 2
Index: 6 | Parks: 5 | Restaurants: 20 | Hospitals: 2
Index: 8 | Parks: 5 | Restaurants: 11 | Hospitals: 4
Index: 12 | Parks: 2 | Restaurants: 10 | Hospitals: 7
Index: 13 | Parks: 3 | Restaurants: 1 | Hospitals: 0
Index: 15 | Parks: 4 | Restaurants: 2 | Hospitals: 0
Index: 16 | Parks: 0 | Restaurants: 2 | Hospitals: 1
Index: 21 | Parks: 2 | Restaurants: 2 | Hospitals: 20
Index: 22 | Parks: 13 | Restaurants: 1 | Hospitals: 3
Index: 23 | Parks: 14 | Restaurants: 20 | Hospitals: 6
Index: 29 | Parks: 1 | Restaurants: 17 | Hospitals: 17
Index: 31 | Parks: 1 | Restaurants: 0 | Hospitals: 0
Index: 51 | Parks: 4 | Restaurants: 13 | Hospitals: 0
Index: 58 | Parks: 1 | Restaurants: 

In [655]:
dfCiudadesFiltCrimWeathCostliveAPI = dfCiudadesFiltCrimWeathCostliveAPI.dropna()
dfCiudadesFiltCrimWeathCostliveAPI.head(45)

,city,lat,lng,country,population,Crime Index,Safety Index,pressure,temp,sea_level,...,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index\t,parks,restaurants,hospitals,aqi,dominant Pollutant
0,Tokyo,35.6897,139.6922,Japan,37732000.0,24.1,75.9,1012.450,302.06875,1012.450,...,29.3,44.1,67.0,35.4,98.6,18.0,6.0,5.0,80.0,pm25
1,Jakarta,-6.1750,106.8275,Indonesia,33756000.0,52.7,47.3,1008.950,302.25975,1008.950,...,12.8,22.9,36.4,16.9,35.5,4.0,0.0,0.0,55.0,pm25
2,Delhi,28.6100,77.2300,India,32226000.0,59.4,40.6,1003.250,305.07975,1003.250,...,7.3,16.3,23.8,19.9,93.9,12.0,2.0,1.0,65.0,pm25
3,Guangzhou,23.1300,113.2600,China,26940000.0,30.4,69.6,1003.175,302.91975,1003.175,...,13.3,24.8,40.2,22.4,70.0,0.0,20.0,20.0,61.0,pm25
4,Mumbai,19.0761,72.8775,India,24973000.0,44.8,55.2,1008.225,300.96550,1008.225,...,19.0,22.2,25.6,21.0,54.2,20.0,20.0,20.0,55.0,pm10
5,Manila,14.5958,120.9772,Philippines,24922000.0,64.7,35.3,1001.450,300.89600,1001.450,...,15.9,27.3,40.5,21.9,29.4,16.0,4.0,2.0,73.0,pm10
6,Shanghai,31.2286,121.4747,China,24073000.0,29.4,70.6,1006.900,301.79400,1006.900,...,30.7,36.7,45.8,26.9,60.4,5.0,20.0,2.0,84.0,pm25
8,Seoul,37.5600,126.9900,"Korea, South",23016000.0,24.5,75.5,1010.950,299.70775,1010.950,...,29.1,50.8,92.1,37.4,91.3,5.0,11.0,4.0,84.0,pm10
12,Dhaka,23.7639,90.3889,Bangladesh,18627000.0,62.7,37.3,1002.650,300.90425,1002.650,...,3.8,16.5,30.8,15.5,27.8,2.0,10.0,7.0,65.0,pm25
13,Beijing,39.9067,116.3975,China,18522000.0,27.0,73.0,1015.775,295.07900,1015.775,...,26.4,32.0,37.8,23.9,82.2,3.0,1.0,0.0,83.0,o3


In [656]:
dfFinal = dfCiudadesFiltCrimWeathCostliveAPI
dfFinal.to_csv('dfFinal.csv',index=False)